### ElasticSearch 

# Analytics

Build an overview of the results that match a query: 
    
Ejemplos de queries: 
- ¿Cuantos comentarios tienen los post que tienen los terminos de búsqueda "Big Data" o "Data Science" ?
- ¿Quienes son los autores más populares de estos términos?
- ¿Cuales son las etiquetas más populares de estos posts? 

We can do these queries to work with near Real Time data 

## Aggregation

* Buckets - collection of documents that match a criterium ==> GROUP BY 
* Metrics - Statistics calculated in a bucket ==> AGGREGATE FUNCTIONS : COUNT(), SUM(), MAX()

* Aggregations are executed in the context of search results - send them to /search 
* It is just another step

## Buckets

 - Definition: A collection of documents that match a criteria
 - Buckets can be nested


## Example : aggregation 

<pre>
GET /cars/transactions/_search
{
    "size" : 0,
    "aggs" : { 
        "popular_colors" : { 
            "terms" : { 
              "field" : "color"
            }
        }
    }
}
</pre>

   * size: 0 indicates that we do not care about the results
   * "popular colors": name of the aggregation 
   

## Example: extract average

<pre>
GET /cars/transactions/_search
{
   "size" : 0,
   "aggs": {
      "colors": {
         "terms": {
            "field": "color"
         },
         "aggs": { 
            "avg_price": { 
               "avg": {
                  "field": "price" 
               }
            }
         }
      }
   }
}
</pre>

## Example: nested aggregations


## Example: histograms

## Example: Time histograms

## Kibana

## Scope

## Filtering aggregations and search

## Sorting

## Aproximate aggregations

 * cardinalities: distinct count
 * percentiles 

## Significant terms